In [1]:
from mxnet import gluon
import mxnet as mx
from mxnet import nd
from mxnet.gluon import nn 
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import PIL.Image

from gluoncv.data import VOCSegmentation
from gluoncv.utils.metrics.voc_segmentation import batch_pix_accuracy, batch_intersection_union

from mylib.deeplabv3p import DeepLabv3p

%matplotlib inline

/home/jiancheng/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:
class CustomVOCSegmentation(VOCSegmentation):
    
    def __init__(self, split, size=(512,512)):
        super().__init__(split=split)
        self.size = size
        
    def __getitem__(self, index):
        img = PIL.Image.open(self.images[index]).convert('RGB')
        mask = PIL.Image.open(self.masks[index])
        img = img.resize(self.size, PIL.Image.BILINEAR)
        mask = mask.resize(self.size, PIL.Image.NEAREST)
        img = self._img_transform(img).transpose((2,0,1))/127.5 - 1.
        mask = self._mask_transform(mask)
        return img, mask

In [3]:
dataset = CustomVOCSegmentation(split='val', size=(512,512))

In [4]:
dataloader = gluon.data.DataLoader(dataset, batch_size=8)

In [5]:
model = DeepLabv3p(OS=8)
model.initialize(ctx=mx.gpu())
# weights = "/home/jiancheng/code/segmentation/deeplabv3p_gluon/tmp_weights/pascal_trainval/pascal_trainval.params"
weights = '/home/jiancheng/code/segmentation/deeplabv3p_gluon/tmp_weights/pascal_train_aug/pascal_train_aug.params'
model.load_params(filename=weights, ctx=mx.gpu())

In [6]:
tbar = tqdm(dataloader)
total_inter, total_union, total_correct, total_label = (0,)*4
for i, (x, y) in enumerate(tbar):
    x = x.copyto(mx.gpu())
    y = y.copyto(mx.gpu())
    pred = model(x)
    correct, labeled = batch_pix_accuracy(output=pred,target=y)
    inter, union = batch_intersection_union(output=pred,target=y,nclass=21)
    total_correct += correct.astype('int64')
    total_label += labeled.astype('int64')
    total_inter += inter.astype('int64')
    total_union += union.astype('int64')
    pix_acc = np.float64(1.0) * total_correct / (np.spacing(1, dtype=np.float64) + total_label)
    IoU = np.float64(1.0) * total_inter / (np.spacing(1, dtype=np.float64) + total_union)
    mIoU = IoU.mean()
    tbar.set_description('iter%s: pix_acc: %.4f, mIoU: %.4f' % (i, pix_acc, mIoU))

iter181: pix_acc: 0.9411, mIoU: 0.7458: 100%|██████████| 182/182 [05:27<00:00,  1.80s/it]


In [7]:
model = DeepLabv3p(OS=16)
model.initialize(ctx=mx.gpu())
# weights = "/home/jiancheng/code/segmentation/deeplabv3p_gluon/tmp_weights/pascal_trainval/pascal_trainval.params"
weights = '/home/jiancheng/code/segmentation/deeplabv3p_gluon/tmp_weights/pascal_train_aug/pascal_train_aug.params'
model.load_params(filename=weights, ctx=mx.gpu())

In [8]:
tbar = tqdm(dataloader)
total_inter, total_union, total_correct, total_label = (0,)*4
for i, (x, y) in enumerate(tbar):
    x = x.copyto(mx.gpu())
    y = y.copyto(mx.gpu())
    pred = model(x)
    correct, labeled = batch_pix_accuracy(output=pred,target=y)
    inter, union = batch_intersection_union(output=pred,target=y,nclass=21)
    total_correct += correct.astype('int64')
    total_label += labeled.astype('int64')
    total_inter += inter.astype('int64')
    total_union += union.astype('int64')
    pix_acc = np.float64(1.0) * total_correct / (np.spacing(1, dtype=np.float64) + total_label)
    IoU = np.float64(1.0) * total_inter / (np.spacing(1, dtype=np.float64) + total_union)
    mIoU = IoU.mean()
    tbar.set_description('iter%s: pix_acc: %.4f, mIoU: %.4f' % (i, pix_acc, mIoU))

iter181: pix_acc: 0.9276, mIoU: 0.7054: 100%|██████████| 182/182 [01:35<00:00,  1.90it/s]


In [9]:
model = DeepLabv3p(OS=8)
model.initialize(ctx=mx.gpu())
weights = "/home/jiancheng/code/segmentation/deeplabv3p_gluon/tmp_weights/pascal_trainval/pascal_trainval.params"
# weights = '/home/jiancheng/code/segmentation/deeplabv3p_gluon/tmp_weights/pascal_train_aug/pascal_train_aug.params'
model.load_params(filename=weights, ctx=mx.gpu())

In [10]:
tbar = tqdm(dataloader)
total_inter, total_union, total_correct, total_label = (0,)*4
for i, (x, y) in enumerate(tbar):
    x = x.copyto(mx.gpu())
    y = y.copyto(mx.gpu())
    pred = model(x)
    correct, labeled = batch_pix_accuracy(output=pred,target=y)
    inter, union = batch_intersection_union(output=pred,target=y,nclass=21)
    total_correct += correct.astype('int64')
    total_label += labeled.astype('int64')
    total_inter += inter.astype('int64')
    total_union += union.astype('int64')
    pix_acc = np.float64(1.0) * total_correct / (np.spacing(1, dtype=np.float64) + total_label)
    IoU = np.float64(1.0) * total_inter / (np.spacing(1, dtype=np.float64) + total_union)
    mIoU = IoU.mean()
    tbar.set_description('iter%s: pix_acc: %.4f, mIoU: %.4f' % (i, pix_acc, mIoU))

iter181: pix_acc: 0.9677, mIoU: 0.8435: 100%|██████████| 182/182 [05:20<00:00,  1.76s/it]


In [11]:
IoU.mean()

0.84350369696449423

In [12]:
np.float64(1.0) * total_inter / (np.spacing(1, dtype=np.float64) + total_union)

array([ 0.96348462,  0.88476809,  0.63088868,  0.82943351,  0.85980665,
        0.65059416,  0.88939677,  0.8669595 ,  0.919627  ,  0.69152122,
        0.95505052,  0.83846504,  0.9273775 ,  0.90485735,  0.86653745,
        0.91636298,  0.68181437,  0.901195  ,  0.7514869 ,  0.89714929,
        0.88680102])

In [13]:
mIoU

0.84350369696449423